# Descipcion variables
## ImageData

### 1. **Condition of the House**
- **Field**: `ImageData.c1c6`
- **Description**: Related to the condition of the house, i.e., "How new and well-maintained it is."
- **Details**: 
  - Score summarizing solely the condition of the property.
  - Scale: 1.0 (Brand New) to 6.0 (Disrepair), with 1 decimal.

### 2. **Quality of the House**
- **Field**: `ImageData.q1q6`
- **Description**: Related to the quality of the house, i.e., "How luxurious or economical are the construction and finishings of the house."
- **Details**: 
  - Score summarizing solely the quality of construction of the property.
  - Scale: 1.0 (Luxury) to 6.0 (Substandard), with 1 decimal.

> **Note**: More information on Condition and Quality can be found [here](#).

---

### 3. **Unique Features Identified**
- **Field**: `ImageData.features_reso`
- **Description**: A list of all the unique features identified in the property by Restb.ai’s RESO feature detection solution.
- **Details**: Features are defined as per the RESO data dictionary.

---

### 4. **Rooms Identified**
- **Field**: `ImageData.room_type_reso`
- **Description**: A list of all the rooms as defined by the RESO data dictionary, identified in the property.

---

### 5. **Exterior Style**
- **Field**: `ImageData.style.exterior`
- **Description**: Top prediction of the exterior style solution.

---

### 6. **Stories/Floors**
- **Field**: `ImageData.style.stories`
- **Description**: Top prediction of the number of stories or floors from the available photos.


## Listing Data

### Characteristics

#### 1. **Lot Features**
- **Field**: `Characteristics.LotFeatures`
- **Description**: A list of features or a description of the lot included in the sale/lease.

#### 2. **Lot Size**
- **Field**: `Characteristics.LotSizeSquareFeet`
- **Description**: The total square footage of the lot.

---

### Listing Details

#### 3. **Close Date**
- **Field**: `Listing.Dates.CloseDate`
- **Description**: The date the purchase agreement was fulfilled.

#### 4. **Listing ID**
- **Field**: `Listing.ListingId`
- **Description**: The well-known identifier for the listing.

#### 5. **Close Price**
- **Field**: `Listing.Price.ClosePrice`
- **Description**: The amount of money paid by the purchaser to the seller for the property under the agreement.

---

### Location Information

#### 6. **City**
- **Field**: `Location.Address.City`
- **Description**: The city in the listing address.

#### 7. **County or Parish**
- **Field**: `Location.Address.CountyOrParish`
- **Description**: The county, parish, or other regional authority.

#### 8. **Postal Code**
- **Field**: `Location.Address.PostalCode`
- **Description**: The postal code portion of a street or mailing address.

#### 9. **Postal Code Plus 4**
- **Field**: `Location.Address.PostalCodePlus4`
- **Description**: The last four digits of a nine-digit U.S. ZIP Code.

#### 10. **State or Province**
- **Field**: `Location.Address.StateOrProvince`
- **Description**: Text field containing the accepted postal abbreviation for the state or province.

#### 11. **Street Direction Prefix**
- **Field**: `Location.Address.StreetDirectionPrefix`
- **Description**: The direction indicator that precedes the listed property's street name.

#### 12. **Street Direction Suffix**
- **Field**: `Location.Address.StreetDirectionSuffix`
- **Description**: The direction indicator that follows the listed property's street address.

#### 13. **Street Name**
- **Field**: `Location.Address.StreetName`
- **Description**: The street name portion of a listed property's street address.

#### 14. **Street Number**
- **Field**: `Location.Address.StreetNumber`
- **Description**: The street number portion of a listed property's street address.

#### 15. **Street Suffix**
- **Field**: `Location.Address.StreetSuffix`
- **Description**: The suffix portion of a listed property's street address.

#### 16. **Unit Number**
- **Field**: `Location.Address.UnitNumber`
#- **Description**: A text field containing the unit number or portion of a larger building or complex (e.g., Apt G, #55).

#### 17. **Unparsed Address**
- **Field**: `Location.Address.UnparsedAddress`
- **Description**: A text representation of the address with the full civic location as a single entity.

#### 18. **Subdivision Name**
- **Field**: `Location.Area.SubdivisionName`
- **Description**: A neighborhood, community, complex, or builder tract.

#### 19. **High School District**
- **Field**: `Location.School.HighSchoolDistrict`
- **Description**: The name of the high school district having a catchment area that includes the associated property.

---

### Property Information

#### 20. **Property Type**
- **Field**: `Property.PropertyType`
- **Description**: A list of types of properties (e.g., Residential, Lease, Income, Land, Mobile, Commercial Sale).

---

### Structure Information

#### 21. **Basement**
- **Field**: `Structure.Basement`
- **Description**: A list of information and features about the basement (e.g., None/Slab, Finished, Crawl Space).

#### 22. **Full Bathrooms**
- **Field**: `Structure.BathroomsFull`
- **Description**: A room containing all four elements of a bath: toilet, sink, bathtub, shower head.

#### 23. **Half Bathrooms**
- **Field**: `Structure.BathroomsHalf`
- **Description**: A room containing two elements of a bath: toilet and sink.

#### 24. **Total Bedrooms**
- **Field**: `Structure.BedroomsTotal`
- **Description**: The total number of bedrooms in the dwelling.

#### 25. **Below Grade Finished Area**
- **Field**: `Structure.BelowGradeFinishedArea`
- **Description**: The finished area within the structure that is below ground.

#### 26. **Below Grade Unfinished Area**
- **Field**: `Structure.BelowGradeUnfinishedArea`
- **Description**: The unfinished area within the structure that is below ground.

#### 27. **Cooling**
- **Field**: `Structure.Cooling`
- **Description**: A list des cribing the cooling or air conditioning features of the property.

#### 28. **Fireplaces**
- **Field**: `Structure.FireplacesTotal`
- **Description**: Indicates whether or not the property includes a fireplace.

#### 29. **Garage Spaces**
- **Field**: `Structure.GarageSpaces`
- **Description**: The number of spaces in the garage(s).

#### 30. **Heating**
- **Field**: `Structure.Heating`
- **Description**: A list describing the heating features of the property.

#### 31. **Living Area**
- **Field**: `Structure.LivingArea`
- **Description**: The total livable area within the structure.

#### 32. **New Construction**
- **Field**: `Structure.NewConstructionYN`
- **Description**: Indicates whether or not the property is newly constructed.

#### 33. **Parking Features**
- **Field**: `Structure.ParkingFeatures`
- **Description**: A list of features about or a description of the parking included in the sale/lease.

#### 34. **Total Rooms**
- **Field**: `Structure.Rooms.RoomsTotal`
- **Description**: The number of rooms in a dwelling.

#### 35. **Year Built**
- **Field**: `Structure.YearBuilt`
- **Description**: The year the property was first granted an occupancy permit.

---

### Tax Information

#### 36. **Zoning**
- **Field**: `Tax.Zoning`
- **Description**: A division of the city or county into areas of different permissible land uses.

---

### Unit Types

#### 37. **Unit Type**
- **Field**: `UnitTypes.UnitTypeType`
- **Description**: A list of possible unit types (e.g., 1 Bedroom, 2 Bedroom, Studio, Loft, etc.).


## Census Data

#### 1. **Census Block**
- **Field**: `Location.Address.CensusBlock`
- **Description**: The smallest geographic unit used by the United States Census Bureau for tabulation of 100-percent data.

#### 2. **Census Tract**
- **Field**: `Location.Address.CensusTract`
- **Description**: A geographic region defined for the purpose of taking a census.

---

### Geographic Information System (GIS)

#### 3. **Latitude**
- **Field**: `Location.GIS.Latitude`
- **Description**: The geographic latitude of some reference point on the property, specified in degrees and decimal parts.

#### 4. **Longitude**
- **Field**: `Location.GIS.Longitude`
- **Description**: The geographic longitude of some reference point on the property, specified in degrees and decimal parts.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score

sns.set()
pd.set_option('display.precision', 3)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Leemos del csv
df_train = pd.read_csv("../train.csv", header= 0, delimiter = ',')
df_test = pd.read_csv("../test.csv",  header= 0, delimiter = ',')

In [ ]:
# remove spaces and symbols to avoid problems with statsmodel GLM
df_train.columns = [c.lower().strip().replace(' ','_').replace('/','_').replace('-','_') 
                                for c in df_train.columns] 

# change the type of categorical variables into category
categorical_columns = list(df_train.dtypes[df_train.dtypes == 'O'].index.values)
for column in categorical_columns:
    df_train[column] = df_train[column].astype('category')

# peek into the data
df_train.head()

## Estadística descriptiva básica

A continuación vemos como hay una mezcla de variables categóricas y numéricas, y la variable objetivo es: **listing.price.closeprice**

In [ ]:
df_train.info();

In [ ]:
df_train.describe(include='all').T

## Visualización básica

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(10,10))

# We will not plot country because it has too many categories.
for i, c in enumerate(df_train.columns[1:1]):
    ax = axes.reshape(-1)[i]
    if df_train[c].dtype.kind == 'O':
        a = sns.countplot(x=c,data=df_train,ax=ax)
    else:
        b = sns.histplot(x=c,data=df_train,ax=ax, bins=20, kde=True, color="skyblue")
        ax.set_title(c, fontsize=16)
        ax.set_xlabel('Valor', fontsize=14)
        ax.set_ylabel('Frecuencia', fontsize=14)

        mean = df_train[c].mean()
        median = df_train[c].median()
        ax.axvline(mean, color='red', linestyle='dashed', linewidth=1, label='Media')
        ax.legend()

plt.tight_layout()
plt.show();


Ahora que sabemos cómo se ven nuestros datos, es una buena idea verificar cuántos valores perdidos tenemos en cada variable, si es que tenemos.